<table style="border-collapse: collapse;">
  <tr>
    <td style="text-align: left; border: none;">
      <h1 style="font-weight: bold;">California Real Estate Price Prediction</h1>
      <p>Using real estate listings collected in the first 6 months in 2021</p>
    </td>
    <td style="border: none;">
      <img src="images/dataset-cover.jpg" alt="Dataset Cover">
    </td>
  </tr>
</table>



# Exploratory Data Analysis Notebook

### Dataset is from Kaggle: https://www.kaggle.com/datasets/yellowj4acket/real-estate-california/data 
This dataset shows real estate listing for California (US) for the first 6 months of 2021. Prices are listed in USD.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df_orig = pd.read_csv('data/RealEstate_California.csv')

# Make a copy of the dataset on which we can do all the column removals and data cleaning
df = pd.read_csv('data/RealEstate_California.csv')

In [2]:
# Allow the entire width of the dataframe to be displayed
pd.set_option('display.max_colwidth', None)

In [3]:
# Display the first 5 rows
df.head()

,sequence,id,stateId,countyId,cityId,country,datePostedString,is_bankOwned,is_forAuction,event,...,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,0,95717-2087851113,9,77,24895,USA,2021-01-13,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Placer County
1,1,94564-18496265,9,189,36958,USA,2021-07-12,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
2,2,94564-18484475,9,190,36958,USA,2021-07-08,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
3,3,94564-18494835,9,191,36958,USA,2021-07-07,0,0,Listed for sale,...,1,1.0,1,Two Story,0,1,0,0,SINGLE_FAMILY,Contra Costa County
4,4,94564-2069722747,9,192,36958,USA,2021-07-07,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Contra Costa County


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35389 entries, 0 to 35388
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sequence            35389 non-null  int64  
 1   id                  35389 non-null  object 
 2   stateId             35389 non-null  int64  
 3   countyId            35389 non-null  int64  
 4   cityId              35389 non-null  int64  
 5   country             35389 non-null  object 
 6   datePostedString    35386 non-null  object 
 7   is_bankOwned        35389 non-null  int64  
 8   is_forAuction       35389 non-null  int64  
 9   event               35100 non-null  object 
 10  time                35100 non-null  float64
 11  price               35389 non-null  float64
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object 
 14  state               35389 non-null  object 
 15  yearBuilt           35389 non-null  int64  
 16  stre

### There are a total of 35,389 records with 39 columns.
Let's start with dropping the columns that are not useful for analysis

### Columns to be dropped

In [5]:
#Sequence, id are not useful for the analysis so we can drop them
df.drop(['sequence', 'id'], axis=1, inplace=True)

#stateId and state will never change since the data is only from California so we can drop it
df.drop(['stateId', 'state'], axis=1, inplace=True)

#drop the 'city' columns since we have 'cityid'
df.drop('city', axis=1, inplace=True)

#drop the 'county' columns since we have 'countyId'
df.drop('county', axis=1, inplace=True)

#country will never change since the data is only from USA so we can drop it
df.drop('country', axis=1, inplace=True)

#time of listing is not likely to drive the sale price so we can drop it
df.drop('time', axis=1, inplace=True)

#streetAddress is too detailed to perform the analysis so we can drop it
df.drop('streetAddress', axis=1, inplace=True)

#latitude and longitude are not needed since we have city and zip code information that will be more useful
df.drop(['latitude', 'longitude'], axis=1, inplace=True)

#description is too detailed to perform the analysis so we can drop it (NLP analysis can be done on this column if desired. But we will leave it out for now. We can include it in future if high accuracy is not achieved).
#df.drop('description', axis=1, inplace=True)

#currency column has only one value (USD) so we can drop it
df.drop('currency', axis=1, inplace=True)

#### Analysis checkpoint 1 (Drop columns that are apparent for not being used for price prediction):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object 
 7   is_bankOwned        35389 non-null  int64  
 8   is_forAuction       35389 non-null  int64  
 9   event               35100 non-null  object 
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64  
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64
 24  livingAreaValue     35389 non-null  float64
 25  lotAreaUnits        35389 non-null  object 
 26  bathrooms           35389 non-null  float64
 27  bedrooms            35389 non-null  float64
 28  buildingArea        35389 non-null  float64
 29  parking             35389 non-null  int64  
 30  garageSpaces        35389 non-null  float64
 31  hasGarage           35389 non-null  int64  
 32  levels              35389 non-null  object 
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object 
 38  county              35389 non-null  object   x   Drop column
 </pre>

### Analyze the target ('price') column

In [6]:
#Identify value counts for price and print the first few price values in ascending order
df['price'].value_counts().sort_index().head()

price
0.0      319
1.0        1
650.0      2
750.0      1
850.0      1
Name: count, dtype: int64

In [7]:
#Prices for houses are typically never in low values (especially in California). 
# Let's identify what the 'price' and 'description' contains for the prices for the first few records that are organized in the ascending order of price where price is not 0.
df.loc[df['price'] > 0, ['price', 'event', 'homeType']].sort_values('price').head()

,price,event,homeType
23108,1.0,Listed for sale,LOT
8056,650.0,Listing removed,SINGLE_FAMILY
32952,650.0,Listing removed,SINGLE_FAMILY
24874,750.0,Listing removed,CONDO
9896,850.0,Listing removed,SINGLE_FAMILY


In [8]:
# The 'event' seems to contain values like 'Listing removed'. Let's identify the value counts in the 'event' column so we know if we need to drop any rows.
df['event'].value_counts()

event
Listed for sale    24622
Price change        4893
Listing removed     4311
Sold                1268
Listed for rent        5
Pending sale           1
Name: count, dtype: int64

In [9]:
# We can drop the rows where the 'event' is 'Listing removed' since the house is not available for sale anymore. Same goes for 'Listed for rent'. We are only interested in the houses that are available for sale.
df = df.loc[(df['event'] != 'Listing removed') & (df['event'] != 'Listed for rent')]

#### Analyze 'homeType' column

In [10]:
# Let's identify the value counts in the 'homeType' column so we know if we need to drop any rows.
df['homeType'].value_counts()

homeType
SINGLE_FAMILY    20638
LOT               4237
CONDO             3170
MULTI_FAMILY      1913
TOWNHOUSE         1114
APARTMENT            1
Name: count, dtype: int64

#### We will focus only on the 'homeType' column value of 'SINGLE_FAMILY'

#### Based on the above analysis related to price, the following types of records will be dropped:
1. 'price'=0
2. 'event'='Listing removed' OR 'event'='Listed for rent'
3. 'homeType'!='SINGLE_FAMILY'

In [11]:
#Drop records where:
# 1. 'price'=0 OR
# 2. 'event'='Listing removed' OR 'event'='Listed for rent' OR
# 3. 'homeType'='SINGLE_FAMILY'
df = df.loc[(df['price'] > 0) & (df['event'] != 'Listing removed') & (df['event'] != 'Listed for rent') & (df['homeType'] == 'SINGLE_FAMILY')]

df.drop('homeType', axis=1, inplace=True)

In [12]:
#print the records in ascending order of price
df.sort_values('price').head()

,countyId,cityId,datePostedString,is_bankOwned,is_forAuction,event,price,pricePerSquareFoot,yearBuilt,zipcode,...,bedrooms,buildingArea,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed
12984,165276,16764,2021-05-27,0,0,Sold,5800.0,1.0,1975,92808.0,...,7.0,0.0,1,5.0,1,Three Or More-Multi/Split,1,1,0,0
34643,121519,33840,2021-07-10,0,0,Sold,7500.0,2.0,1991,92675.0,...,6.0,0.0,1,4.0,1,Two,1,1,0,0
24714,327763,33840,2021-07-10,0,0,Sold,7500.0,2.0,1991,92675.0,...,6.0,0.0,1,4.0,1,Two,1,1,0,0
31259,75475,36429,2018-11-09,0,0,Sold,20000.0,16.0,1975,93667.0,...,0.0,0.0,0,0.0,0,0,0,0,0,0
19093,249862,20172,2021-07-06,0,0,Sold,22000.0,22.0,1959,93555.0,...,3.0,0.0,1,1.0,1,One,0,0,0,0


#### Analysis checkpoint 2 ('price', 'event', 'homeType'):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object 
 7   is_bankOwned        35389 non-null  int64  
 8   is_forAuction       35389 non-null  int64  
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64  
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64
 24  livingAreaValue     35389 non-null  float64
 25  lotAreaUnits        35389 non-null  object 
 26  bathrooms           35389 non-null  float64
 27  bedrooms            35389 non-null  float64
 28  buildingArea        35389 non-null  float64
 29  parking             35389 non-null  int64  
 30  garageSpaces        35389 non-null  float64
 31  hasGarage           35389 non-null  int64  
 32  levels              35389 non-null  object 
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>


### Analyze 'is_bankOwned' column data

In [13]:
#Check on the number of unique values for is_bankOwned and is_forAuction columns
df['is_bankOwned'].value_counts()

is_bankOwned
0    20369
1        3
Name: count, dtype: int64

##### Since the 'is_bankOwned' has a value of 1 for only 3 records, these records can be dropped since they are far less records to drive any useful predictions.

### Analyze 'is_forAuction' column data

In [14]:
df['is_forAuction'].value_counts()

is_forAuction
0    20363
1        9
Name: count, dtype: int64

##### Since the 'is_forAuction' has a value of 1 for only 9 records, these records can be dropped since they are far less records to drive any useful predictions.

### Analyze 'hasBadGeocode' column data

In [15]:
#check number of unique values for hasBadGeocode column
df['hasBadGeocode'].value_counts()

hasBadGeocode
0    20372
Name: count, dtype: int64

##### Since the 'hasBadGeocode' has a value of 0 only, no further action is needed for this column.

#### Records to be dropped:
- 'is_bankOwned' with value of 1
- 'is_forAuction' with a value of 1
- Drop the three columns ('is_bankOwned', 'is_forAuction', 'hasBadGeocode') since they now have value of only 0.

In [16]:
#Drop records where:
# 'is_bankOwned' with value of 1
# 'is_forAuction' with a value of 1
df = df.loc[(df['is_bankOwned'] == 0) & (df['is_forAuction'] == 0)]

#Drop the columns 'is_bankOwned', 'is_forAuction', 'hasBadGeocode' since they have only one value
df.drop(['is_bankOwned', 'is_forAuction', 'hasBadGeocode'], axis=1, inplace=True)

#### Analysis checkpoint 3 ('is_bankOwned', 'is_forAuction', 'hasBadGeocode'):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object 
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64
 24  livingAreaValue     35389 non-null  float64
 25  lotAreaUnits        35389 non-null  object 
 26  bathrooms           35389 non-null  float64
 27  bedrooms            35389 non-null  float64
 28  buildingArea        35389 non-null  float64
 29  parking             35389 non-null  int64  
 30  garageSpaces        35389 non-null  float64
 31  hasGarage           35389 non-null  int64  
 32  levels              35389 non-null  object 
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20360 entries, 1 to 35388
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            20360 non-null  int64  
 1   cityId              20360 non-null  int64  
 2   datePostedString    20360 non-null  object 
 3   event               20360 non-null  object 
 4   price               20360 non-null  float64
 5   pricePerSquareFoot  20360 non-null  float64
 6   yearBuilt           20360 non-null  int64  
 7   zipcode             20360 non-null  float64
 8   description         20259 non-null  object 
 9   livingArea          20360 non-null  float64
 10  livingAreaValue     20360 non-null  float64
 11  lotAreaUnits        20360 non-null  object 
 12  bathrooms           20360 non-null  float64
 13  bedrooms            20360 non-null  float64
 14  buildingArea        20360 non-null  float64
 15  parking             20360 non-null  int64  
 16  garageSpa

#### Analyze the bathrooms and bedrooms since we are focused on homes that are liveable and ensure they have some good values in order to design a better performing model

In [18]:
#Perform value count on 'bathrooms' column and list them in the ascending order
df['bathrooms'].value_counts().sort_index()

bathrooms
0.0      262
1.0     2510
2.0     8121
3.0     6274
4.0     1710
5.0      804
6.0      337
7.0      147
8.0      102
9.0       38
10.0      19
11.0      16
12.0       9
13.0       1
14.0       5
15.0       2
20.0       1
25.0       1
58.0       1
Name: count, dtype: int64

In [19]:
# Perform value count on 'bedrooms' column and list them in the ascending order
df['bedrooms'].value_counts().sort_index()

bedrooms
0.0       73
1.0      245
2.0     2461
3.0     8569
4.0     6235
5.0     2093
6.0      444
7.0      140
8.0       53
9.0       24
10.0       6
11.0       6
12.0       3
13.0       3
15.0       1
17.0       1
32.0       2
52.0       1
Name: count, dtype: int64

In [20]:
#Considering that we are focusing solely on SINGLE_FAMILY homes, it is unlikely that these will have 0 bedrooms or 0 bathrooms. 
# We will treat them as records with missing values and drop such records.

# Drop records where 'bedrooms' is 0 or 'bathrooms' is 0
df = df.loc[(df['bedrooms'] > 0) & (df['bathrooms'] > 0)]

#### Analysis checkpoint 4 (bedrooms, bathrooms):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object 
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64
 24  livingAreaValue     35389 non-null  float64
 25  lotAreaUnits        35389 non-null  object 
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64
 29  parking             35389 non-null  int64  
 30  garageSpaces        35389 non-null  float64
 31  hasGarage           35389 non-null  int64  
 32  levels              35389 non-null  object 
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20079 entries, 1 to 35388
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            20079 non-null  int64  
 1   cityId              20079 non-null  int64  
 2   datePostedString    20079 non-null  object 
 3   event               20079 non-null  object 
 4   price               20079 non-null  float64
 5   pricePerSquareFoot  20079 non-null  float64
 6   yearBuilt           20079 non-null  int64  
 7   zipcode             20079 non-null  float64
 8   description         19979 non-null  object 
 9   livingArea          20079 non-null  float64
 10  livingAreaValue     20079 non-null  float64
 11  lotAreaUnits        20079 non-null  object 
 12  bathrooms           20079 non-null  float64
 13  bedrooms            20079 non-null  float64
 14  buildingArea        20079 non-null  float64
 15  parking             20079 non-null  int64  
 16  garageSpa

### Analyze 'livingArea' and 'livingAreaValue' columns data

In [22]:
#check number of unique values for livingArea column
df['livingArea'].value_counts()

livingArea
0.0       195
1200.0     93
1500.0     75
1400.0     62
2000.0     60
         ... 
3835.0      1
4376.0      1
498.0       1
7360.0      1
4385.0      1
Name: count, Length: 3963, dtype: int64

In [23]:
#check number of unique values for livingAreaValue column
df['livingAreaValue'].value_counts()

livingAreaValue
0.0       194
1200.0     93
1500.0     75
1400.0     62
2000.0     60
         ... 
3835.0      1
4376.0      1
498.0       1
7360.0      1
4385.0      1
Name: count, Length: 3964, dtype: int64

In [24]:
#Idenitfy the number of records where 'livingArea' and 'livingAreaValue' are not equal
df.loc[df['livingArea'] != df['livingAreaValue']].shape

(1, 23)

##### 1. Since the 'livingArea' and 'livingAreaValue' seem identical, one of the columns ('livingArea') can be dropped. 
##### 2. Drop the records with livingAreaValue=0.

In [25]:
#Drop the 'livingArea' column 
df.drop('livingArea', axis=1, inplace=True)

#drop records with 'livingAreaValue'=0
df = df.loc[df['livingAreaValue'] > 0]

#### Analysis checkpoint 5 (livingArea, livingAreaValue):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object 
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64  x   Drop column
 24  livingAreaValue     35389 non-null  float64  x   Drop records where livingAreaValue=0
 25  lotAreaUnits        35389 non-null  object 
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64
 29  parking             35389 non-null  int64  
 30  garageSpaces        35389 non-null  float64
 31  hasGarage           35389 non-null  int64  
 32  levels              35389 non-null  object 
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19885 entries, 1 to 35388
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            19885 non-null  int64  
 1   cityId              19885 non-null  int64  
 2   datePostedString    19885 non-null  object 
 3   event               19885 non-null  object 
 4   price               19885 non-null  float64
 5   pricePerSquareFoot  19885 non-null  float64
 6   yearBuilt           19885 non-null  int64  
 7   zipcode             19885 non-null  float64
 8   description         19785 non-null  object 
 9   livingAreaValue     19885 non-null  float64
 10  lotAreaUnits        19885 non-null  object 
 11  bathrooms           19885 non-null  float64
 12  bedrooms            19885 non-null  float64
 13  buildingArea        19885 non-null  float64
 14  parking             19885 non-null  int64  
 15  garageSpaces        19885 non-null  float64
 16  hasGarage

#### Analyze the 'datePostedString' column

In [27]:
# analyze the 'datePostedString' column in the descending order of 'datePostedString'
df['datePostedString'].sort_values(ascending=False)

17628    2021-07-14
17049    2021-07-14
19205    2021-07-14
19196    2021-07-14
17085    2021-07-14
            ...    
8567     2019-04-02
12205    2019-01-28
23369    2018-11-11
25186    2018-10-25
26807    2017-12-08
Name: datePostedString, Length: 19885, dtype: object

In [28]:
# Since the intent is to not include a timeseries analysis, let's drop the data for anything older than 2021-01-01
df = df.loc[df['datePostedString'] >= '2021-01-01']

#create a new column 'month' from the 'datePostedString' column
df['month'] = pd.to_datetime(df['datePostedString']).dt.month

#convert 'month' to a object variable so it can be used as a categorical variable
df['month'] = df['month'].astype('object')

In [29]:
# We can then drop the 'datePostedString' column since it is no longer needed
df.drop('datePostedString', axis=1, inplace=True)

#### Analysis checkpoint 6 (datePostedString):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object   x   Drop all records older than '2021-01-01', then drop this column
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64  x   Drop column
 24  livingAreaValue     35389 non-null  float64  x   Drop records where livingAreaValue=0
 25  lotAreaUnits        35389 non-null  object 
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64
 29  parking             35389 non-null  int64  
 30  garageSpaces        35389 non-null  float64
 31  hasGarage           35389 non-null  int64  
 32  levels              35389 non-null  object 
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19764 entries, 1 to 35388
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            19764 non-null  int64  
 1   cityId              19764 non-null  int64  
 2   event               19764 non-null  object 
 3   price               19764 non-null  float64
 4   pricePerSquareFoot  19764 non-null  float64
 5   yearBuilt           19764 non-null  int64  
 6   zipcode             19764 non-null  float64
 7   description         19664 non-null  object 
 8   livingAreaValue     19764 non-null  float64
 9   lotAreaUnits        19764 non-null  object 
 10  bathrooms           19764 non-null  float64
 11  bedrooms            19764 non-null  float64
 12  buildingArea        19764 non-null  float64
 13  parking             19764 non-null  int64  
 14  garageSpaces        19764 non-null  float64
 15  hasGarage           19764 non-null  int64  
 16  levels   

#### Analyze the 'buildingArea' column

In [31]:
#Identify value counts for 'buildingArea' column and list them in the ascending order
df['buildingArea'].value_counts().sort_index()

buildingArea
0.0          15983
1.0              2
300.0            1
324.0            1
400.0            1
             ...  
20000.0          2
20125.0          1
21407.0          1
692604.0         1
9061351.0        1
Name: count, Length: 1960, dtype: int64

#### Since a large majority of 'buildingArea' values are 0 (almost 16K out of the total 19.7K total records), this column is not useful for any price predictions. This column can be dropped.

In [32]:
#drop 'buildingArea' column
df.drop('buildingArea', axis=1, inplace=True)

#### Analyze the 'lotAreaUnits' column

In [33]:
#Identify value counts for 'lotAreaUnits' column and list them in the ascending order
df['lotAreaUnits'].value_counts().sort_index()

lotAreaUnits
Acres     5939
sqft     13825
Name: count, dtype: int64

#### 'lotAreaUnits' does not have a corresponding column (like 'lotArea' or 'buildingArea') to be associated with. As such this column can be dropped.

In [34]:
#drop 'lotAreaUnits' column
df.drop('lotAreaUnits', axis=1, inplace=True)

#### Analysis checkpoint 7 (buildingArea, lotAreaUnits):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object   x   Drop all records older than '2021-01-01', then drop this column
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64  x   Drop column
 24  livingAreaValue     35389 non-null  float64  x   Drop records where livingAreaValue=0
 25  lotAreaUnits        35389 non-null  object   x   Drop column
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64  x   Drop column
 29  parking             35389 non-null  int64  
 30  garageSpaces        35389 non-null  float64
 31  hasGarage           35389 non-null  int64  
 32  levels              35389 non-null  object 
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19764 entries, 1 to 35388
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            19764 non-null  int64  
 1   cityId              19764 non-null  int64  
 2   event               19764 non-null  object 
 3   price               19764 non-null  float64
 4   pricePerSquareFoot  19764 non-null  float64
 5   yearBuilt           19764 non-null  int64  
 6   zipcode             19764 non-null  float64
 7   description         19664 non-null  object 
 8   livingAreaValue     19764 non-null  float64
 9   bathrooms           19764 non-null  float64
 10  bedrooms            19764 non-null  float64
 11  parking             19764 non-null  int64  
 12  garageSpaces        19764 non-null  float64
 13  hasGarage           19764 non-null  int64  
 14  levels              19764 non-null  object 
 15  pool                19764 non-null  int64  
 16  spa      

#### Analyze parking related columns
1. 'garageSpaces' column
2. 'hasGarage' column
3. 'parking' column

In [36]:
#check if hasGarage column has value of 0 when garageSpaces is not 0. Get a count of such records.
df.loc[(df['garageSpaces'] == 0) & (df['hasGarage'] >0)].shape

#Remove such records
df = df.loc[(df['garageSpaces'] > 0) | (df['hasGarage'] == 0)]

In [37]:
#Check the other way around now. Check if hasGarage column has value of 0 when garageSpaces > 0. Get a count of such records.
df.loc[(df['garageSpaces'] > 0) & (df['hasGarage'] == 0)].shape

#Remove such records
df = df.loc[(df['garageSpaces'] == 0) | (df['hasGarage'] > 0)]

In [38]:
#Now 'hasGarage' column is not needed since we have 'garageSpaces' column
df.drop('hasGarage', axis=1, inplace=True)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19228 entries, 1 to 35388
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            19228 non-null  int64  
 1   cityId              19228 non-null  int64  
 2   event               19228 non-null  object 
 3   price               19228 non-null  float64
 4   pricePerSquareFoot  19228 non-null  float64
 5   yearBuilt           19228 non-null  int64  
 6   zipcode             19228 non-null  float64
 7   description         19130 non-null  object 
 8   livingAreaValue     19228 non-null  float64
 9   bathrooms           19228 non-null  float64
 10  bedrooms            19228 non-null  float64
 11  parking             19228 non-null  int64  
 12  garageSpaces        19228 non-null  float64
 13  levels              19228 non-null  object 
 14  pool                19228 non-null  int64  
 15  spa                 19228 non-null  int64  
 16  isNewCons

In [40]:
#get the value counts for 'parking' column
df['parking'].value_counts()

parking
1    15701
0     3527
Name: count, dtype: int64

In [41]:
#Is 'parking' ever 0 when 'garageSpaces' is not 0?
df.loc[(df['garageSpaces'] == 0) & (df['parking'] >0)].shape

(4401, 19)

##### Retain records that have (df['garageSpaces'] > 0) | (df['hasGarage'] == 0) and  (df['garageSpaces'] == 0) | (df['hasGarage'] > 0), then drop 'hasGarage' column
##### Retain 'parking' as-is since parking flag is independent of 'garageSpaces'

#### Analysis checkpoint 8 (garageSpaces, hasGarage, parking):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object   x   Drop all records older than '2021-01-01', then drop this column
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64  x   Drop column
 24  livingAreaValue     35389 non-null  float64  x   Drop records where livingAreaValue=0
 25  lotAreaUnits        35389 non-null  object   x   Drop column
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64  x   Drop column
 29  parking             35389 non-null  int64    x   Retain as-is
 30  garageSpaces        35389 non-null  float64  x   Retain records that have (df['garageSpaces'] > 0) | (df['hasGarage'] == 0) and  (df['garageSpaces'] == 0) | (df['hasGarage'] > 0), then drop 'hasGarage' column
 31  hasGarage           35389 non-null  int64    x   (review comment for 'garageSpaces' column)
 32  levels              35389 non-null  object 
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19228 entries, 1 to 35388
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            19228 non-null  int64  
 1   cityId              19228 non-null  int64  
 2   event               19228 non-null  object 
 3   price               19228 non-null  float64
 4   pricePerSquareFoot  19228 non-null  float64
 5   yearBuilt           19228 non-null  int64  
 6   zipcode             19228 non-null  float64
 7   description         19130 non-null  object 
 8   livingAreaValue     19228 non-null  float64
 9   bathrooms           19228 non-null  float64
 10  bedrooms            19228 non-null  float64
 11  parking             19228 non-null  int64  
 12  garageSpaces        19228 non-null  float64
 13  levels              19228 non-null  object 
 14  pool                19228 non-null  int64  
 15  spa                 19228 non-null  int64  
 16  isNewCons

##### Analyze 'levels'

In [43]:
# analyze 'levels' column in the ascending order of 'levels'
df['levels'].value_counts().sort_index()

levels
0                                      7107
1                                        39
2                                        41
3                                        15
4+                                        2
Multi/Split                             269
Multi/Split-One                           2
Multi/Split-Three Or More                 2
Multi/Split-Tri-Level                     3
Multi/Split-Two                          10
One                                    6225
One Story                               576
One Story-One                             5
One-Multi/Split                           9
One-Three Or More                         2
One-Two                                  44
One-Two-Multi/Split                       2
One-Two-Three Or More                     1
Other                                     9
Split Level                               2
Three                                    11
Three Or More                           330
Three Or More-Multi/Split

##### 'levels' has many values but while there many unique values, they can be cleaned up to consolidate into a fewer set of values:
- '1', 'One Story', 'One Story-One' = 'One'
- '2', 'Two Story', 'Two Story-One', 'Two Story-Two',  'Two-One' = 'Two'
- '3', '4+', 'Three', 'Multi/Split-One', 'Multi/Split-Three Or More', 'Multi/Split-Tri-Level', 'Multi/Split-Two, 'One-Multi/Split', 'One-Three Or More', 'One-Two', 'One-Two-Multi/Split', 'One-Two-Three Or More', 'Split Level', 'Three Or More-Multi/Split', 'Three Or More-Two', 'Three or More Stories', 'Three or More Stories-One-Two', 'Three or More Stories-Three Or More', 'Tri-Level',  'Two-Multi/Split', 'Two-Three Or More' = 'Multi/Split'

In [44]:
# Update the values in the 'levels' such that:
# '0', '1', 'One Story', 'One Story-One' = 'One'
# '2', 'Two Story', 'Two Story-One', 'Two Story-Two',  'Two-One' = 'Two'
# '3', '4+', 'Three', 'Three Or More', 'Multi/Split-One', 'Multi/Split-Three Or More', 'Multi/Split-Tri-Level', 'Multi/Split-Two, 'One-Multi/Split', 'One-Three Or More', 'One-Two', 'One-Two-Multi/Split', 'One-Two-Three Or More', 'Split Level', 'Three Or More-Multi/Split', 'Three Or More-Two', 'Three or More Stories', 'Three or More Stories-One-Two', 'Three or More Stories-Three Or More', 'Tri-Level',  'Two-Multi/Split', 'Two-Three Or More' = 'Multi/Split'

df['levels'] = df['levels'].replace(['0', '1', 'One Story', 'One Story-One'], 'One') #assuming '0' means 'One' for single story houses
df['levels'] = df['levels'].replace(['2', 'Two Story', 'Two Story-One', 'Two Story-Two', 'Two-One'], 'Two')
df['levels'] = df['levels'].replace(['3', '4+', 'Three', 'Three Or More', 'Multi/Split-One', 'Multi/Split-Three Or More', 'Multi/Split-Tri-Level', 'Multi/Split-Two', 'One-Multi/Split', 'One-Three Or More', 'One-Two', 'One-Two-Multi/Split', 'One-Two-Three Or More', 'Split Level', 'Three Or More-Multi/Split', 'Three Or More-Two', 'Three or More Stories', 'Three or More Stories-One-Two', 'Three or More Stories-Three Or More', 'Tri-Level',  'Two-Multi/Split', 'Two-Three Or More'], 'Multi/Split')

In [45]:
#print records with 'levels'='Other' in ascending order of 'price'
df.loc[df['levels'] == 'Other'].sort_values('price')

,countyId,cityId,event,price,pricePerSquareFoot,yearBuilt,zipcode,description,livingAreaValue,bathrooms,bedrooms,parking,garageSpaces,levels,pool,spa,isNewConstruction,hasPetsAllowed,month
17618,228861,12804,Price change,369000.0,503.0,1964,95462.0,Open House: Saturday July 17th 12-2pm. Priced to sell. SELLER IS MOTIVATED! Call Danielle Collard 510-502-8246 for easy showings. Charming home is nestled among the Redwoods on a spacious corner lot above the Russian River. Needs a bit of TLC. Relax or entertain in the welcoming family room with brick fireplace. Laundry & living area all on one level. Kitchen has Quartz countertops undermount sink & wood cabinets. Large Bedroom has mirrored closet doors separate sitting area & lots of windows with amazing views of the Redwood forest. The Office/2nd bedroom has plenty of windows & private balcony. The laundry room has full size washer dryer & small freezer. The large basement is partially finished with plenty of storage. One car garage/work shop with storage & a large driveway. Additional parking areas.,734.0,1.0,1.0,1,1.0,Other,0,0,0,0,4
32639,95085,13072,Listed for sale,699000.0,462.0,1914,94603.0,Come and see this home. Big back yard. Needs TLC bring in your contractor. Public record shows this is a( Duplex 2 units).,1512.0,2.0,4.0,0,0.0,Other,0,0,0,0,6
16592,216754,13072,Listed for sale,749000.0,654.0,1924,94619.0,Don’t miss this charming 2 bed/1 bath bungalow with add’l bonus suite in popular Maxwell Park! Set up on a hill with a sunny location & loads of light you’ll love its attractive front presence with a landscaped yard and a welcoming front porch. Charm abounds with its original details: the fireplace and built-in cabinets recessed ceilings molding and handsome hardwood flooring. The living & dining rooms look out to pleasant hillside views. Updates include newer windows recessed lighting & an updated kitchen & with granite counters and stainless appliances. On the main level enjoy two bedrooms a full bath & a huge walk-in closet for a potential primary bath. A separate studio with full bath is perfect for a large office or in-law suite. Relax in the peaceful backyard with mature landscaping two patios & grass for entertaining and play. Loads of off-street parking or a play area on the extra long driveway. Close to Laurel district shops restaurants freeways & transportation.,1145.0,2.0,2.0,0,0.0,Other,0,0,0,0,7
16747,219670,13072,Listed for sale,789000.0,595.0,1912,94607.0,"""One of a kind traditional urban conversion with charming corner bay window.The best of both worlds! Two fantastic spaces with separate entrances.Upon entering the 1326 SqFt. up-stairs unit you will feel welcomed by the spacious open layout captivating high ceilings & natural filtered light throughout that highlights the original Douglas Fir floors.Bedrooms with ample closet space and an attic that you must see to believe with potential to be converted into a loft. All tastefully renovated with professional craftsmanship! Ground level space has two separate entrances that will welcome the investor in you into a magnificent 1024 SqFt. white canvas which will spark your imagination with its 10 high ceiling and exposed re-enforced structure. An absolute $$$+ making opportunity. Lovely yard partly shaded by a majestic avocado tree. Operable gate providing opportunity for gated parking.Great location!Minutes to Oak. airport SF Berkeley Bart other. Easy access to HWY 80/580 & 24.""",1326.0,2.0,3.0,0,0.0,Other,0,0,0,0,7
13194,166905,13699,Price change,1499000.0,769.0,1946,94401.0,Pack your belongings and move in. Beautifully remodeled home located in the city of San Mateo. Home features family room living room 3 bedrooms 2 full bathrooms recessed lights elegant engineered hardwood floors dual pane windows and freshly painted inside and outside. Huge private master suite with mirror closet door. Cook your next meal in an exquisite kitchen with brand new cabinets Korean counter-tops stainless steel appliances a

##### It does not look like the 'Other' value can be consolidated into another 'levels' value since the types and prices of homes vary quite a bit.

In [46]:
df['levels'].value_counts().sort_index()

levels
Multi/Split      846
One            13952
Other              9
Two             4421
Name: count, dtype: int64

#### Analysis checkpoint 9 (levels):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object   x   Drop all records older than '2021-01-01', then drop this column
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64  x   Drop column
 24  livingAreaValue     35389 non-null  float64  x   Drop records where livingAreaValue=0
 25  lotAreaUnits        35389 non-null  object   x   Drop column
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64  x   Drop column
 29  parking             35389 non-null  int64    x   Retain as-is
 30  garageSpaces        35389 non-null  float64  x   Retain records that have (df['garageSpaces'] > 0) | (df['hasGarage'] == 0) and  (df['garageSpaces'] == 0) | (df['hasGarage'] > 0), then drop 'hasGarage' column
 31  hasGarage           35389 non-null  int64    x   (review comment for 'garageSpaces' column)
 32  levels              35389 non-null  object   x   (See comment at the base of this cell)
 33  pool                35389 non-null  int64  
 34  spa                 35389 non-null  int64  
 35  isNewConstruction   35389 non-null  int64  
 36  hasPetsAllowed      35389 non-null  int64  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>

Update the values in the 'levels' such that:
- '0', '1', 'One Story', 'One Story-One' = 'One'
- '2', 'Two Story', 'Two Story-One', 'Two Story-Two',  'Two-One' = 'Two'
- '3', '4+', 'Three', 'Three Or More', 'Multi/Split-One', 'Multi/Split-Three Or More', 'Multi/Split-Tri-Level', 'Multi/Split-Two, 'One-Multi/Split', 'One-Three Or More', 'One-Two', 'One-Two-Multi/Split', 'One-Two-Three Or More', 'Split Level', 'Three Or More-Multi/Split', 'Three Or More-Two', 'Three or More Stories', 'Three or More Stories-One-Two', 'Three or More Stories-Three Or More', 'Tri-Level',  'Two-Multi/Split', 'Two-Three Or More' = 'Multi/Split'


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19228 entries, 1 to 35388
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            19228 non-null  int64  
 1   cityId              19228 non-null  int64  
 2   event               19228 non-null  object 
 3   price               19228 non-null  float64
 4   pricePerSquareFoot  19228 non-null  float64
 5   yearBuilt           19228 non-null  int64  
 6   zipcode             19228 non-null  float64
 7   description         19130 non-null  object 
 8   livingAreaValue     19228 non-null  float64
 9   bathrooms           19228 non-null  float64
 10  bedrooms            19228 non-null  float64
 11  parking             19228 non-null  int64  
 12  garageSpaces        19228 non-null  float64
 13  levels              19228 non-null  object 
 14  pool                19228 non-null  int64  
 15  spa                 19228 non-null  int64  
 16  isNewCons

##### Analyze 'pool', 'spa', 'isNewConstruction' and 'hasPetsAllowed' columns

In [48]:
#analyze 'pool' column
df['pool'].value_counts()

pool
0    16081
1     3147
Name: count, dtype: int64

In [49]:
#analyze 'spa' column
df['spa'].value_counts()

spa
0    15391
1     3837
Name: count, dtype: int64

In [50]:
#analyze 'isNewConstruction' column
df['isNewConstruction'].value_counts()

isNewConstruction
0    18959
1      269
Name: count, dtype: int64

In [51]:
#analyze 'hasPetsAllowed' column
df['hasPetsAllowed'].value_counts()

hasPetsAllowed
0    18972
1      256
Name: count, dtype: int64

##### Since the number of records where 'hasPetsAllowed' is so low (and it seems counter to the fact no permission is needed for pets when you own a single family home in California), this column can be dropped as '0' in this case indicates that it has not been properly filled.

In [52]:
#drop 'hasPetsAllowed' column
df.drop('hasPetsAllowed', axis=1, inplace=True)

#### Analysis checkpoint 10 ('pool', 'spa', 'isNewConstruction' and 'hasPetsAllowed' columns):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Retain as-is
 4   cityId              35389 non-null  int64    x   Retain as-is
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object   x   Drop all records older than '2021-01-01', then drop this column
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64  
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64  x   Drop column
 24  livingAreaValue     35389 non-null  float64  x   Drop records where livingAreaValue=0
 25  lotAreaUnits        35389 non-null  object   x   Drop column
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64  x   Drop column
 29  parking             35389 non-null  int64    x   Retain as-is
 30  garageSpaces        35389 non-null  float64  x   Retain records that have (df['garageSpaces'] > 0) | (df['hasGarage'] == 0) and  (df['garageSpaces'] == 0) | (df['hasGarage'] > 0), then drop 'hasGarage' column
 31  hasGarage           35389 non-null  int64    x   (review comment for 'garageSpaces' column)
 32  levels              35389 non-null  object   x   (See comment at the base of this cell)
 33  pool                35389 non-null  int64    x   Retain as-is
 34  spa                 35389 non-null  int64    x   Retain as-is  
 35  isNewConstruction   35389 non-null  int64    x   Retain as-is    
 36  hasPetsAllowed      35389 non-null  int64    x   Drop column  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>

Update the values in the 'levels' such that:
- '0', '1', 'One Story', 'One Story-One' = 'One'
- '2', 'Two Story', 'Two Story-One', 'Two Story-Two',  'Two-One' = 'Two'
- '3', '4+', 'Three', 'Three Or More', 'Multi/Split-One', 'Multi/Split-Three Or More', 'Multi/Split-Tri-Level', 'Multi/Split-Two, 'One-Multi/Split', 'One-Three Or More', 'One-Two', 'One-Two-Multi/Split', 'One-Two-Three Or More', 'Split Level', 'Three Or More-Multi/Split', 'Three Or More-Two', 'Three or More Stories', 'Three or More Stories-One-Two', 'Three or More Stories-Three Or More', 'Tri-Level',  'Two-Multi/Split', 'Two-Three Or More' = 'Multi/Split'


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19228 entries, 1 to 35388
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countyId            19228 non-null  int64  
 1   cityId              19228 non-null  int64  
 2   event               19228 non-null  object 
 3   price               19228 non-null  float64
 4   pricePerSquareFoot  19228 non-null  float64
 5   yearBuilt           19228 non-null  int64  
 6   zipcode             19228 non-null  float64
 7   description         19130 non-null  object 
 8   livingAreaValue     19228 non-null  float64
 9   bathrooms           19228 non-null  float64
 10  bedrooms            19228 non-null  float64
 11  parking             19228 non-null  int64  
 12  garageSpaces        19228 non-null  float64
 13  levels              19228 non-null  object 
 14  pool                19228 non-null  int64  
 15  spa                 19228 non-null  int64  
 16  isNewCons

In [54]:
# Count missing values in 'zipcode'
missing_zipcodes = df['zipcode'].isnull().sum()

print(f"Number of missing zip codes: {missing_zipcodes}")

Number of missing zip codes: 0


#### Convert columns to categorical if they are not meant to be numeric (like countyId, cityId, yearBuilt, zipcode)

In [55]:
#Convert numerical columns to categorical columns since they are not continuous: countyId, cityId, yearBuilt, zipcode
# df['countyId'] = df['countyId'].astype('object')
# df['cityId'] = df['cityId'].astype('object')
df['zipcode'] = df['zipcode'].astype('object')

df['yearBuilt'] = df['yearBuilt'].astype('object')


In [56]:
#Drop countyId and cityId columns since we have zipcode column which is more useful
df.drop(['countyId', 'cityId'], axis=1, inplace=True)

#### Convert binary integer columns to bool values

In [57]:
# Ensure columns are initially of integer type (0/1)
df['parking'] = df['parking'].astype(int)
df['pool'] = df['pool'].astype(int)
df['spa'] = df['spa'].astype(int)
df['isNewConstruction'] = df['isNewConstruction'].astype(int)

# Convert integer columns to boolean (True/False)
df['parking'] = df['parking'].astype(bool)
df['pool'] = df['pool'].astype(bool)
df['spa'] = df['spa'].astype(bool)
df['isNewConstruction'] = df['isNewConstruction'].astype(bool)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19228 entries, 1 to 35388
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   event               19228 non-null  object 
 1   price               19228 non-null  float64
 2   pricePerSquareFoot  19228 non-null  float64
 3   yearBuilt           19228 non-null  object 
 4   zipcode             19228 non-null  object 
 5   description         19130 non-null  object 
 6   livingAreaValue     19228 non-null  float64
 7   bathrooms           19228 non-null  float64
 8   bedrooms            19228 non-null  float64
 9   parking             19228 non-null  bool   
 10  garageSpaces        19228 non-null  float64
 11  levels              19228 non-null  object 
 12  pool                19228 non-null  bool   
 13  spa                 19228 non-null  bool   
 14  isNewConstruction   19228 non-null  bool   
 15  month               19228 non-null  object 
dtypes: bool(4

#### Analysis checkpoint 11 (Drop: 'countyId', 'cityId'; Convert to categorical columns: 'yearBuilt', 'zipcode'; Convert 'parking', 'pool', 'spa', 'isNewConstruction' to bool):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Drop column
 4   cityId              35389 non-null  int64    x   Drop column
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object   x   Drop all records older than '2021-01-01', then drop this column
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64  
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64    x   Convert to categorical column
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64  x   Convert to categorical column
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Drop column
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64  x   Drop column
 24  livingAreaValue     35389 non-null  float64  x   Drop records where livingAreaValue=0
 25  lotAreaUnits        35389 non-null  object   x   Drop column
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64  x   Drop column
 29  parking             35389 non-null  int64    x   Convert to bool
 30  garageSpaces        35389 non-null  float64  x   Retain records that have (df['garageSpaces'] > 0) | (df['hasGarage'] == 0) and  (df['garageSpaces'] == 0) | (df['hasGarage'] > 0), then drop 'hasGarage' column
 31  hasGarage           35389 non-null  int64    x   (review comment for 'garageSpaces' column)
 32  levels              35389 non-null  object   x   (See comment at the base of this cell)
 33  pool                35389 non-null  int64    x   Convert to bool
 34  spa                 35389 non-null  int64    x   Convert to bool
 35  isNewConstruction   35389 non-null  int64    x   Convert to bool
 36  hasPetsAllowed      35389 non-null  int64    x   Drop column  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>

Update the values in the 'levels' such that:
- '0', '1', 'One Story', 'One Story-One' = 'One'
- '2', 'Two Story', 'Two Story-One', 'Two Story-Two',  'Two-One' = 'Two'
- '3', '4+', 'Three', 'Three Or More', 'Multi/Split-One', 'Multi/Split-Three Or More', 'Multi/Split-Tri-Level', 'Multi/Split-Two, 'One-Multi/Split', 'One-Three Or More', 'One-Two', 'One-Two-Multi/Split', 'One-Two-Three Or More', 'Split Level', 'Three Or More-Multi/Split', 'Three Or More-Two', 'Three or More Stories', 'Three or More Stories-One-Two', 'Three or More Stories-Three Or More', 'Tri-Level',  'Two-Multi/Split', 'Two-Three Or More' = 'Multi/Split'


#### Analyze 'description'
We will use description for performing property description analysis using NLP.

In [59]:
#Identify the number of records with null values for description column
missing_description = df['description'].isnull().sum()
print(f"Number of missing descriptions: {missing_description}")

Number of missing descriptions: 98


In [60]:
#drop the records with missing description
df = df.dropna(subset=['description'])

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19130 entries, 1 to 35388
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   event               19130 non-null  object 
 1   price               19130 non-null  float64
 2   pricePerSquareFoot  19130 non-null  float64
 3   yearBuilt           19130 non-null  object 
 4   zipcode             19130 non-null  object 
 5   description         19130 non-null  object 
 6   livingAreaValue     19130 non-null  float64
 7   bathrooms           19130 non-null  float64
 8   bedrooms            19130 non-null  float64
 9   parking             19130 non-null  bool   
 10  garageSpaces        19130 non-null  float64
 11  levels              19130 non-null  object 
 12  pool                19130 non-null  bool   
 13  spa                 19130 non-null  bool   
 14  isNewConstruction   19130 non-null  bool   
 15  month               19130 non-null  object 
dtypes: bool(4

#### Analyze "pricePerSquareFoot"

In [62]:
#Perform correlation analysis on pricePerSquareFoot and price
df[['price', 'pricePerSquareFoot']].corr()

,price,pricePerSquareFoot
price,1.000000,0.018704
pricePerSquareFoot,0.018704,1.000000


##### The correlation coefficient between price and pricePerSquareFoot of 0.018741 indicates a very weak positive correlation. This means there's almost no linear relationship between the two variables.

As such, I am going to exclude pricePerSquareFoot from the ML model for these reasons:
1. Redundancy: The information contained in pricePerSquareFoot is already captured by livingArea and price. pricePerSquareFoot is simply calculated from these two.
2. Potential for Bias: Including pricePerSquareFoot might introduce bias into the model, particularly if the dataset contains outliers or unusual pricing practices.
3. Model Complexity: Adding an unnecessary feature increases model complexity and can make it harder to interpret.

In [63]:
#drop 'pricePerSquareFoot' column since it has a weak correlation with 'price'
df.drop('pricePerSquareFoot', axis=1, inplace=True)

#### Analysis checkpoint 12 (pricePerSquareFoot):
<pre>
 0   sequence            35389 non-null  int64    x   Drop column
 1   id                  35389 non-null  object   x   Drop column
 2   stateId             35389 non-null  int64    x   Drop column
 3   countyId            35389 non-null  int64    x   Drop column
 4   cityId              35389 non-null  int64    x   Drop column
 5   country             35389 non-null  object   x   Drop column
 6   datePostedString    35386 non-null  object   x   Drop all records older than '2021-01-01', then drop this column
 7   is_bankOwned        35389 non-null  int64    x   Drop records where value is 1 then drop this column
 8   is_forAuction       35389 non-null  int64    x   Drop records where value is 1 then drop this column
 9   event               35100 non-null  object   x   Drop records where 'event'='Listing removed' OR 'event'='Listed for rent'
 10  time                35100 non-null  float64  x   Drop column
 11  price               35389 non-null  float64  x   Drop records where price=0
 12  pricePerSquareFoot  35389 non-null  float64  x   Drop this column because of a very weak correlation with price.
 13  city                35389 non-null  object   x   Drop column
 14  state               35389 non-null  object   x   Drop column
 15  yearBuilt           35389 non-null  int64    x   Convert to categorical column
 16  streetAddress       35388 non-null  object   x   Drop column
 17  zipcode             35364 non-null  float64  x   Convert to categorical column
 18  longitude           35389 non-null  float64  x   Drop column
 19  latitude            35389 non-null  float64  x   Drop column
 20  hasBadGeocode       35389 non-null  int64    x   Drop column
 21  description         35110 non-null  object   x   Retain as-is.
 22  currency            35389 non-null  object   x   Drop column
 23  livingArea          35389 non-null  float64  x   Drop column
 24  livingAreaValue     35389 non-null  float64  x   Drop records where livingAreaValue=0
 25  lotAreaUnits        35389 non-null  object   x   Drop column
 26  bathrooms           35389 non-null  float64  x   Drop records where bathrooms=0
 27  bedrooms            35389 non-null  float64  x   Drop records where bedrooms=0
 28  buildingArea        35389 non-null  float64  x   Drop column
 29  parking             35389 non-null  int64    x   Convert to bool
 30  garageSpaces        35389 non-null  float64  x   Retain records that have (df['garageSpaces'] > 0) | (df['hasGarage'] == 0) and  (df['garageSpaces'] == 0) | (df['hasGarage'] > 0), then drop 'hasGarage' column
 31  hasGarage           35389 non-null  int64    x   (review comment for 'garageSpaces' column)
 32  levels              35389 non-null  object   x   (See comment at the base of this cell)
 33  pool                35389 non-null  int64    x   Convert to bool
 34  spa                 35389 non-null  int64    x   Convert to bool
 35  isNewConstruction   35389 non-null  int64    x   Convert to bool 
 36  hasPetsAllowed      35389 non-null  int64    x   Drop column  
 37  homeType            35389 non-null  object   x   Retain records where 'homeType'='SINGLE_FAMILY' and then drop this column 
 38  county              35389 non-null  object   x   Drop column
 </pre>

Update the values in the 'levels' such that:
- '0', '1', 'One Story', 'One Story-One' = 'One'
- '2', 'Two Story', 'Two Story-One', 'Two Story-Two',  'Two-One' = 'Two'
- '3', '4+', 'Three', 'Three Or More', 'Multi/Split-One', 'Multi/Split-Three Or More', 'Multi/Split-Tri-Level', 'Multi/Split-Two, 'One-Multi/Split', 'One-Three Or More', 'One-Two', 'One-Two-Multi/Split', 'One-Two-Three Or More', 'Split Level', 'Three Or More-Multi/Split', 'Three Or More-Two', 'Three or More Stories', 'Three or More Stories-One-Two', 'Three or More Stories-Three Or More', 'Tri-Level',  'Two-Multi/Split', 'Two-Three Or More' = 'Multi/Split'


### Final state of dataframe to be used for ML modeling

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19130 entries, 1 to 35388
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   event              19130 non-null  object 
 1   price              19130 non-null  float64
 2   yearBuilt          19130 non-null  object 
 3   zipcode            19130 non-null  object 
 4   description        19130 non-null  object 
 5   livingAreaValue    19130 non-null  float64
 6   bathrooms          19130 non-null  float64
 7   bedrooms           19130 non-null  float64
 8   parking            19130 non-null  bool   
 9   garageSpaces       19130 non-null  float64
 10  levels             19130 non-null  object 
 11  pool               19130 non-null  bool   
 12  spa                19130 non-null  bool   
 13  isNewConstruction  19130 non-null  bool   
 14  month              19130 non-null  object 
dtypes: bool(4), float64(5), object(6)
memory usage: 1.8+ MB
